# production

> Contains the production flow.

In [ ]:
#| default_exp production

/Users/kobus/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx
/Users/kobus/aishiftscheduler/aishiftscheduler/best_theta_data.xlsx
/Users/kobus/aishiftscheduler/aishiftscheduler/AIShiftScheduler1.png

len(thNAMES)=5
thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect']


In [ ]:
!which python

/Users/kobus/aishiftscheduler/.venv/bin/python


In [ ]:
!python --version

Python 3.10.0


In [ ]:
# Set up autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from copy import copy
import time
import math
from pprint import pprint
import ray
import json
from io import StringIO
import aishiftscheduler.loader as ldr
import aishiftscheduler.config as cf
import aishiftscheduler.visualization as vis
import aishiftscheduler.model as mod
import aishiftscheduler.policy as pol
import aishiftscheduler.inferencer as inf
import aishiftscheduler.trainer as trn
import aishiftscheduler.evaluator as evl
import aishiftscheduler.parameters as par
from PIL import Image
import aishiftscheduler.utils as utl
from fastapi import FastAPI
from pydantic import BaseModel

In [ ]:
np.__version__, pd.__version__, mpl.__version__, ray.__version__, json.__version__

('1.26.4', '2.0.3', '3.8.3', '2.9.3', '2.0.9')

In [ ]:
!pip list|grep fastapi

fastapi                   0.110.0


In [ ]:
!pip list|grep uvicorn

uvicorn                   0.27.1


In [ ]:
#| export
def prepare_schedule(pars):
    start = time.time()
    if 'TRAIN' in cf.MODES:
        print('############################## TRAIN ##############################')
        L = 2 #5 #20 #10 #5 #2 #3 #2db #10pub
        T = 7*pars.SLOTS_PER_DAY #5 #7*96
        First_n_t = int(1.2*T)
        Last_n_t = int(1*T) ##make whole multiple of T to look better in chart
        
        trn_Best_theta, trn_Worst_theta, trn_Df_first_n_t, trn_Df_last_n_t = \
            trn.train_schedule(L, T, First_n_t, Last_n_t, pars)
        
        trn_Best_theta_df = pd.DataFrame({
            'thCumSlots': [trn_Best_theta[0]], 
            'thSickProb': [trn_Best_theta[1]], 
            'thCumMerits': [trn_Best_theta[2]], 
            'thContSlots': [trn_Best_theta[3]],
            'thSelect': [trn_Best_theta[4]]
        })
        ldr.save_best_theta(cf.PATH_best_theta_data, trn_Best_theta_df)
        print(f'{trn_Best_theta=}')
        print(f'{trn_Df_first_n_t.shape=}')

    if 'EVALU' in cf.MODES:
        print('############################## EVALU ##############################')
        ## COMMENTED OUT TEMPORARILY FOR QUICKER TESTING
        # L = 2 #20 #5 #2 #2db #10pub
        # T = 7*pars.SLOTS_PER_DAY #=672
        # First_n_t = int(.11*L*T)
        
        # evl_Best_theta_opt, evl_Worst_theta_opt, evl_Df_opt = \
        #     evl.evalu_schedule_opt(L, T, First_n_t, trn_Best_theta, pars)
        # print(f'{evl_Best_theta_opt=}')
        # print(f'{evl_Df_opt.shape=}')
    
        # evl_Best_theta_non, evl_Worst_theta_non, evl_Df_non = \
        #     evl.evalu_schedule_non(L, T, First_n_t, trn_Worst_theta, pars)
        # print(f'{evl_Best_theta_non=}')
        # print(f'{evl_Df_non.shape=}')

        # ## single sample-path
        # L = 1 #20 #5 #2 #2db #10pub
        # T = 7*pars.SLOTS_PER_DAY #=672
        # First_n_t = int(1*L*T)
        
        # evl_Best_theta_opt, evl_Worst_theta_opt, evl_Df_opt = \
        #     evl.evalu_schedule_opt(L, T, First_n_t, trn_Best_theta, pars)
        # print(f'{evl_Best_theta_opt=}')
        # print(f'{evl_Df_opt.shape=}')
    
        # evl_Best_theta_non, evl_Worst_theta_non, evl_Df_non = \
        #     evl.evalu_schedule_non(L, T, First_n_t, trn_Worst_theta, pars)
        # print(f'{evl_Best_theta_non=}')
        # print(f'{evl_Df_non.shape=}')

        # print(evl_Df_opt[evl_Df_opt['t']==T-1][['Ccum']])
        # print(evl_Df_non[evl_Df_non['t']==T-1][['Ccum']])
        
    if 'INFER' in cf.MODES:
        print('############################## INFER ##############################')
        L = 1
        T = 7*pars.SLOTS_PER_DAY
        First_n_t = int(1*L*T)
    
        # stored_best_theta = inf.get_best_theta_Alloc()
        stored_best_theta_tuple = ldr.load_best_theta(cf.PATH_best_theta_data)
        stored_best_theta = pol.Policy(None).build_theta({
            'thCumSlots': stored_best_theta_tuple[0],
            'thSickProb': stored_best_theta_tuple[1],
            'thCumMerits': stored_best_theta_tuple[2],
            'thContSlots': stored_best_theta_tuple[3],
            'thSelect': stored_best_theta_tuple[4]
            })
        
        inf_Best_theta_evalu_opt, inf_Df_evalu_opt, prepped_sched = \
            inf.infer_schedule(L, T, First_n_t, stored_best_theta, pars)
        print(f'{inf_Best_theta_evalu_opt=}')
        print(f'{inf_Df_evalu_opt.shape=}')
        # print(f'{prepped_sched=}')
        utl.print_schedule_shifts(inf_Df_evalu_opt, pars)
        # utl.print_schedule_slots(inf_Df_evalu_opt, pars)
    end = time.time(); print(f'EXECUTION TIME: {end - start} seconds')
    return prepped_sched

/Users/kobus/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx
/Users/kobus/aishiftscheduler/aishiftscheduler/best_theta_data.xlsx
/Users/kobus/aishiftscheduler/aishiftscheduler/AIShiftScheduler1.png

len(thNAMES)=5
thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect']


In [ ]:
#| export
# Update the Parameters instance, `Pars` with the latest user input.
# def update_parameters_from_user_input(
def update_parameters_from_user_input(pars, user_input):
    # pars,
    # start, 
    # slots_per_day, 
    # max_daily_slot_run, 
    # resources, 
    # demands_per_busyness, 
    # demands_per_volume, 
    # demands_per_revenue, 
    # resource_expenses):
  error = None 
  pars.START_DATE_TIME = user_input.start
  sd = pd.to_datetime(user_input.start)
  if not sd.strftime('%a')=='Mon':
    error = f"ERROR: {user_input.start} is a {sd.strftime('%A')}. It should be a Monday."
    print(error)
    return error
  for_now_sd = pd.to_datetime('2023-12-04')
  if not sd==for_now_sd:
    error = f"ERROR: Start date must be {for_now_sd} for now."
    print(error) 
    return error

  spd = int(user_input.slots_per_day)
  if not spd == 24:
    error = f"ERROR: Slots per day must be 24 for now."
    print(error)
    return error
  pars.SLOTS_PER_DAY = spd
  
  mdsr = int(user_input.max_daily_slot_run)
  if not mdsr <= spd:
    error = f"ERROR: 'Max daily slot run' must be less than or equal to 'Slots per day'"
    print(error)
    return error
  pars.MAX_DAILY_SLOT_RUN = mdsr

  resource_type_and_ids = user_input.resources.split(';')
  resource_types = []
  resource_ids = []
  resource_type_counts = []
  for itm in resource_type_and_ids:
      res_type, res_ids = itm.split(':')
      resource_types.append(res_type.strip())
      sep_ids = res_ids.split(','); ##print(f'{sep_ids=}')
      for rid in sep_ids:
          resource_ids.append(rid.strip())
      resource_type_counts.append(len(sep_ids))
  print(f'{resource_types=}')
  print(f'{resource_type_counts=}')
  print(f'{resource_ids=}')
  if len(resource_types) > cf.MAX_RESOURCE_TYPES:
    error = f"ERROR: The number of resource types should not exceed {cf.MAX_RESOURCE_TYPES}.\nYou entered the following resource types: {resource_types}"
    print(error)
    return error
  if len(resource_ids) > cf.MAX_RESOURCE_IDS:
    error = f"ERROR: The number of resources should not exceed {cf.MAX_RESOURCE_IDS}.\nYou entered the following resources: {resource_ids}"
    print(error)
    return error
  pars.RESOURCE_TYPES = resource_types; print(f'{pars.RESOURCE_TYPES=}')
  pars.RESOURCE_TYPE_COUNTS = resource_type_counts; print(f'{pars.RESOURCE_TYPE_COUNTS=}')
  pars.TYPES = []
  for i in range(len(pars.RESOURCE_TYPES)):
    additional_types = [pars.RESOURCE_TYPES[i]]*pars.RESOURCE_TYPE_COUNTS[i]
    for item in additional_types:
      pars.TYPES.append(item)
  print(f'{pars.TYPES=}')
  pars.RESOURCE_IDS = resource_ids; print(f'{pars.RESOURCE_TYPE_COUNTS=}')

  rates = user_input.demands_per_busyness.split(',')
  if '' in rates:
    error = f"ERROR: There should be a demand-per-busyness for each resource type (role).\nYou entered the following values: {demands_per_busyness}"
    print(error)
    return error
  rates = list(map(lambda x: float(x), rates))
  print(f'{rates=}')
  if len(rates) != len(pars.RESOURCE_TYPES):
    error = f"ERROR: The number of demands-per-busyness should be the same as the number of resource types (roles).\nYou entered the following values: {demands_per_busyness}"
    print(error)
    return error
  pars.DEMANDS_PER_BUSYNESS = rates
  pars.DEMAND_PER_BUSYNESS = {e: pars.DEMANDS_PER_BUSYNESS[i] for i,e in enumerate(pars.RESOURCE_TYPES)}

  rates = user_input.demands_per_volume.split(',')
  if '' in rates:
    error = f"ERROR: There should be a demand-per-volume for each resource type (role).\nYou entered the following values: {demands_per_volume}"
    print(error)
    return error
  rates = list(map(lambda x: float(x), rates))
  print(f'{rates=}')
  if len(rates) != len(pars.RESOURCE_TYPES):
    error = f"ERROR: The number of demands-per-volume should be the same as the number of resource types (roles).\nYou entered the following values: {demands_per_volume}"
    print(error)
    return error
  pars.DEMANDS_PER_VOLUME = rates
  pars.DEMAND_PER_VOLUME = {e: pars.DEMANDS_PER_VOLUME[i] for i,e in enumerate(pars.RESOURCE_TYPES)}

  rates = user_input.demands_per_revenue.split(',')
  if '' in rates:
    error = f"ERROR: There should be a demand-per-revenue for each resource type (role).\nYou entered the following values: {demands_per_revenue}"
    print(error)
    return error
  rates = list(map(lambda x: float(x), rates))
  print(f'{rates=}')
  if len(rates) != len(pars.RESOURCE_TYPES):
    error = f"ERROR: The number of demands-per-revenue should be the same as the number of resource types (roles).\nYou entered the following values: {demands_per_revenue}"
    print(error)
    return error
  pars.DEMANDS_PER_REVENUE = rates
  pars.DEMAND_PER_REVENUE = {e: pars.DEMANDS_PER_REVENUE[i] for i,e in enumerate(pars.RESOURCE_TYPES)}

  expenses = user_input.resource_expenses.split(',')
  if '' in expenses:
    error = f"ERROR: There should be a resource expense for each resource type (role).\nYou entered the following values: {resource_expenses}"
    print(error)
    return error  
  expenses = list(map(lambda x: float(x), expenses))
  print(f'{expenses=}')
  if len(expenses) != len(pars.RESOURCE_TYPES):
    error = f"ERROR: The number of resource expenses should be the same as the number of resource types (roles).\nYou entered the following values: {resource_expenses}"
    print(error)
    return error
  pars.RESOURCE_EXPENSES = expenses
  pars.RESOURCE_EXPENSE = {e: pars.RESOURCE_EXPENSES[i] for i,e in enumerate(pars.RESOURCE_TYPES)}

  pars.aNAMES = [tup[0]+'_'+tup[1] for tup in zip(pars.TYPES, pars.RESOURCE_IDS)]; print(f'{pars.aNAMES=}')
  pars.bNAMES = pars.RESOURCE_TYPES; print(f'{pars.bNAMES=}')
  pars.abNAMES = [] ##to DEMAND b
  for a in pars.aNAMES:
    a0,a1 = a.split('_')
    for b in pars.bNAMES:
      if(a0==b):
        abn = (a + '___' + b)
        pars.abNAMES.append(abn)
  print(f'{pars.abNAMES=}')
  pars.LABELS = pars.setup_plot_labels()
  return error

In [ ]:
#| export
class UserInput(BaseModel):
    start: str
    slots_per_day: int
    max_daily_slot_run: int
    resources: str
    demands_per_busyness: str
    demands_per_volume: str
    demands_per_revenue: str
    resource_expenses: str

In [ ]:
# ############################################################
# #################### FLOW STARTS ###########################
# ############################################################

In [ ]:
#| export
app = FastAPI()

In [ ]:
#| export
@app.get("/")
def root():
    return {"message": "Hello Oupa ..."}

In [ ]:
#| export
dui = UserInput(
  start="2023-12-04", #2023-12-11
  slots_per_day=24,
  max_daily_slot_run=9, #8
  resources="Manager: Matt; AssistMngr: Mike, Tanner; RetailAssoc: Jake, James, Jane, John, Jim, Jenny, Jeremy, Judy, Julie, Jeffrey",
  # resources = 'Manager: Matt; AssistMngr: Mike, Tanner; RetailAssoc: Jake, James, Jane, John, Jim, Jenny, Jeremy, Judy, Julie',
  # resources = 'Manager: John, Penelope; SalesPerson: Sally, Sarah, Jim, Costa',
  # resources = 'SupChief: Ruan, Francine; Sup: Azra, Wendie, Penny, Sally',
  # resources = 'ChiefTeller: Ruan, Francine; Teller: Azra, Wendie, Penny, Sally',

  # demands_per_busyness = '.2, 4',
  demands_per_busyness="0.005, 0.008, 0.02",

  # demands_per_volume = '.02, .4',
  demands_per_volume="0.03, 0.08, 0.2",

  # demands_per_revenue = '.05, .8',
  demands_per_revenue=".00005, 0.0001, 0.0008",

  # resource_expenses = '35.17, 23.85'
  resource_expenses="25.0, 20.0, 18.0"    
)

In [ ]:
#| export
@app.get("/defaultuserinput")
def get_default_user_input():
    return {
        "start": dui.start,
        "slots_per_day": dui.slots_per_day,
        "max_daily_slot_run": dui.max_daily_slot_run,
        "resources": dui.resources,
        "demands_per_busyness": dui.demands_per_busyness,
        "demands_per_volume": dui.demands_per_volume,
        "demands_per_revenue": dui.demands_per_revenue,
        "resource_expenses": dui.resource_expenses
    }

In [ ]:
#| export
# Create a Parameter instance & initialize with default pars.
# The `Pars` instance will be passed between various modules
Pars = par.Parameters()


len(self.RESOURCE_TYPES)=3
['Manager', 'AssistMngr', 'RetailAssoc']

len(self.TYPES)=13
['Manager', 'AssistMngr', 'AssistMngr', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc']
len(self.aNAMES)=13
['Manager_Matt', 'AssistMngr_Mike', 'AssistMngr_Tanner', 'RetailAssoc_Jake', 'RetailAssoc_James', 'RetailAssoc_Jane', 'RetailAssoc_John', 'RetailAssoc_Jim', 'RetailAssoc_Jenny', 'RetailAssoc_Jeremy', 'RetailAssoc_Judy', 'RetailAssoc_Julie', 'RetailAssoc_Jeffrey']

len(self.bNAMES)=3
['Manager', 'AssistMngr', 'RetailAssoc']

len(self.abNAMES)=13
['Manager_Matt___Manager', 'AssistMngr_Mike___AssistMngr', 'AssistMngr_Tanner___AssistMngr', 'RetailAssoc_Jake___RetailAssoc', 'RetailAssoc_James___RetailAssoc', 'RetailAssoc_Jane___RetailAssoc', 'RetailAssoc_John___RetailAssoc', 'RetailAssoc_Jim___RetailAssoc', 'RetailAssoc_Jenny___RetailAssoc', 'RetailAssoc_Jeremy___RetailAssoc', 'RetailAssoc_Judy___

In [ ]:
# ///////////////////////////////

In [ ]:
# prepare_schedule(Pars)

/Users/kobus/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx
/Users/kobus/aishiftscheduler/aishiftscheduler/best_theta_data.xlsx
/Users/kobus/aishiftscheduler/aishiftscheduler/AIShiftScheduler1.png

len(thNAMES)=5
thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect']


In [ ]:
# \\\\\

In [ ]:
#| export
@app.post("/schedule")
def find_schedule(user_input: UserInput):
    error = update_parameters_from_user_input(Pars, user_input)
    if error == None:
        sched = prepare_schedule(Pars)
        # prepare_schedule(Pars)
        return sched
    else:
        return error

In [ ]:
## test
# prepare_schedule(Pars)

############################## TRAIN ##############################


2024-03-04 04:47:38,507	INFO worker.py:1724 -- Started a local Ray instance.


numThetas=8
... printing every 1th theta (if considered) ...
(pid=51561) /Users/kobus/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx
(pid=51561) /Users/kobus/aishiftscheduler/aishiftscheduler/AIShiftScheduler1.png
(pid=51561) thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect']
(parallel_run_grid_sample_paths pid=51561) 	%%% l=1
(pid=51569) /Users/kobus/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx [repeated 7x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(pid=51569) len(thNAMES)=5 [repeated 23x across cluster]
(pid=51569) thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect'] [repeated 7x across cluster]
(parallel_run_grid_sample_paths pid=51565) 	%%% l=2 [repeated 8x across cluster]
0/7, 443,818, Theta(thCumSlots=0.0, thSickProb=0, thCumMerits=0

/Users/kobus/aishiftscheduler/aishiftscheduler/policy.py:212: RuntimeWarning: invalid value encountered in scalar divide
  Ctilcum_tmp = np.sum(np.square(np.array(CcumIomega__lI) - Cbarcum_tmp))/(L - 1)


"SCHEDULE SHIFTS:\n================\n\n************** Manager_Matt:\nMon Dec 04 09h00\nMon Dec 04 17h00\n\nTue Dec 05 09h00\nTue Dec 05 17h00\n\nWed Dec 06 09h00\nWed Dec 06 17h00\n\nThu Dec 07 09h00\nThu Dec 07 17h00\n\nFri Dec 08 09h00\nFri Dec 08 17h00\n\nSat Dec 09 09h00\nSat Dec 09 17h00\n\nSun Dec 10 09h00\nSun Dec 10 17h00\n\n************** AssistMngr_Mike:\nMon Dec 04 17h00\nMon Dec 04 21h00\n\nTue Dec 05 17h00\nTue Dec 05 21h00\n\nWed Dec 06 17h00\nWed Dec 06 21h00\n\nThu Dec 07 16h00\nThu Dec 07 21h00\n\nFri Dec 08 09h00\nFri Dec 08 17h00\n\nSat Dec 09 09h00\nSat Dec 09 17h00\n\nSun Dec 10 14h00\nSun Dec 10 19h00\n\n************** AssistMngr_Tanner:\nMon Dec 04 09h00\nMon Dec 04 17h00\n\nTue Dec 05 09h00\nTue Dec 05 17h00\n\nWed Dec 06 09h00\nWed Dec 06 17h00\n\nThu Dec 07 09h00\nThu Dec 07 17h00\n\nFri Dec 08 17h00\nFri Dec 08 21h00\n\nSat Dec 09 13h00\nSat Dec 09 20h00\n\nSun Dec 10 09h00\nSun Dec 10 17h00\n\n************** RetailAssoc_Jake:\nMon Dec 04 15h00\nMon Dec 04 20

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

/Users/kobus/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx
/Users/kobus/aishiftscheduler/aishiftscheduler/AIShiftScheduler1.png

len(thNAMES)=5
thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect']
